In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
# Change directory to the folder containing your dataset
dataset_path = '/content/drive/MyDrive/Skin_Cancer_Dataset'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define image size and batch size
image_size = (224, 224)
batch_size = 32
# Create ImageDataGenerator for train set with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Create ImageDataGenerator for test and validation sets without augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
dataset_path

'/content/drive/MyDrive/Skin_Cancer_Dataset'

In [ ]:
# Load train set images
import os

# Define file paths
train_dir = os.path.join(dataset_path, 'train')

# Load train set images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 1934 images belonging to 2 classes.


In [ ]:
# Load test set images
test_dir = os.path.join(dataset_path, 'test')

# Load train set images
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Found 932 images belonging to 2 classes.


In [ ]:
# Load validation set images
val_dir = os.path.join(dataset_path, 'val')
val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)
val_generator

Found 1864 images belonging to 2 classes.


In [ ]:
labels = ['benign', 'malignant']
label_np=np.array(labels)

In [ ]:
# Create CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(generator= train_generator, 
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

<ipython-input-32-65e06b05411d>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator= train_generator,


Epoch 1/10
60/60 [==============================] - 1437s 23s/step - loss: 0.0113 - accuracy: 0.9942 - val_loss: 293.5085 - val_accuracy: 0.5242
Epoch 2/10
60/60 [==============================] - 338s 6s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 295.5364 - val_accuracy: 0.5242
Epoch 3/10
60/60 [==============================] - 333s 6s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 295.0945 - val_accuracy: 0.5248
Epoch 4/10
60/60 [==============================] - 330s 6s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 296.2719 - val_accuracy: 0.5226
Epoch 5/10
60/60 [==============================] - 314s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 295.6127 - val_accuracy: 0.5237
Epoch 6/10
60/60 [==============================] - 314s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 295.6314 - val_accuracy: 0.5237
Epoch 7/10
60/60 [==============================] - 338s 6s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 295.7